## Curse of dimensionality

In [1]:
import numpy as np
from numpy.linalg import norm
import pandas as pd
import matplotlib.pyplot 

In [ ]:
def Lp_dist():
    pass

In [4]:
# dimensions range
dimensions = list(range(2,101))
# data points
N = 100
# number of data sets
q = 100
# p distance measure range
p_range = [0.5, 1, 2, 5, np.inf]


# Set up containers for statistic values
# Relative contrast
ctr = [0]*len(dimensions)
# Average Minimum Distance
avg_min_dist = [0]*len(p_range)
# Average Maximum Distance
avg_max_dist = [0]*len(p_range)
# Average Mean Distance
avg_mean_dist = [0]*len(p_range)
# Average Variance
avg_variance = [0]*len(p_range)

# Doing tests
for i,k in enumerate(dimensions):
    for _ in q:
        # generate random data set in
        # the half-open interval [0,1)
        data = np.random.uniform(low=0, high=1, size=(N, k))

        for p in p_range:

    

i: 0 | k: 2
i: 1 | k: 3
i: 2 | k: 4
i: 3 | k: 5
i: 4 | k: 6
i: 5 | k: 7
i: 6 | k: 8
i: 7 | k: 9
i: 8 | k: 10
i: 9 | k: 11
i: 10 | k: 12
i: 11 | k: 13
i: 12 | k: 14
i: 13 | k: 15
i: 14 | k: 16
i: 15 | k: 17
i: 16 | k: 18
i: 17 | k: 19
i: 18 | k: 20
i: 19 | k: 21
i: 20 | k: 22
i: 21 | k: 23
i: 22 | k: 24
i: 23 | k: 25
i: 24 | k: 26
i: 25 | k: 27
i: 26 | k: 28
i: 27 | k: 29
i: 28 | k: 30
i: 29 | k: 31
i: 30 | k: 32
i: 31 | k: 33
i: 32 | k: 34
i: 33 | k: 35
i: 34 | k: 36
i: 35 | k: 37
i: 36 | k: 38
i: 37 | k: 39
i: 38 | k: 40
i: 39 | k: 41
i: 40 | k: 42
i: 41 | k: 43
i: 42 | k: 44
i: 43 | k: 45
i: 44 | k: 46
i: 45 | k: 47
i: 46 | k: 48
i: 47 | k: 49
i: 48 | k: 50
i: 49 | k: 51
i: 50 | k: 52
i: 51 | k: 53
i: 52 | k: 54
i: 53 | k: 55
i: 54 | k: 56
i: 55 | k: 57
i: 56 | k: 58
i: 57 | k: 59
i: 58 | k: 60
i: 59 | k: 61
i: 60 | k: 62
i: 61 | k: 63
i: 62 | k: 64
i: 63 | k: 65
i: 64 | k: 66
i: 65 | k: 67
i: 66 | k: 68
i: 67 | k: 69
i: 68 | k: 70
i: 69 | k: 71
i: 70 | k: 72
i: 71 | k: 73
i: 72 | k: